In [6]:
!pip install fair-esm openpyxl scikit-learn pandas numpy torch xlrd

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
     |████████████████████████████████| 96 kB 65.0 MB/s eta 0:00:01
You should consider upgrading via the '/home/ma-user/anaconda3/envs/PyTorch-1.8/bin/python3.7 -m pip install --upgrade pip' command.


In [3]:
!pip show torch

Name: torch
Version: 1.8.0
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /home/ma-user/anaconda3/envs/PyTorch-1.8/lib/python3.7/site-packages
Requires: numpy, typing-extensions
Required-by: torchvision, torchtext


# **第一步：准备工作台！(环境设置与导入库)**

想象一下，我们要开始一个重要的生物实验！首先得把实验台收拾干净，把所有需要的工具和试剂准备好，对吧？这一步就是做这个准备工作。我们会：

1.  **导入“工具箱” :** 加载所有需要的 Python 库，比如 `pandas` (处理表格数据超方便)、`numpy` (科学计算基础)、`torch` 和 `esm` (我们请来的“蛋白质语言大师” ESM 模型就住在这里面)、以及 `sklearn` (机器学习“武器库”，随机森林就在这)。
2.  **设定“实验参数” :** 告诉程序我们的数据放在哪里 (`DATA_DIR`)，要用哪个版本的 ESM 模型 (`ESM_MODEL_NAME` - 我们会选一个对 CPU 友好的版本哦！)，以及比赛规则（比如最多只能突变几个位点 `MAX_MUTATIONS`）。
3.  **检查“硬件”:** 看看有没有 GPU 可用。

In [4]:
import pandas as pd  # 处理表格数据（如 Excel、CSV）
import numpy as np   # 数学计算库
import torch         # PyTorch，主要用于模型加载与运行
import esm           # Facebook 的蛋白语言模型库 ESM（用于提取蛋白序列嵌入）
import os            # 操作文件路径
import random        # 设置随机种子
from sklearn.model_selection import train_test_split  # 训练集测试集划分
from sklearn.metrics import r2_score                  # 模型性能评估指标
import re           # 正则表达式，用于处理突变信息
import warnings
warnings.filterwarnings('ignore')  # 忽略警告信息（不影响主逻辑）

# 常量定义 
TRAIN_DATA_FILE = os.path.join('GFP_data.xlsx')  
# 包含亮度、突变信息等训练数据的 Excel 文件
WT_SEQ_FILE = os.path.join('AAseqs of 4 GFP proteins.txt')  
# 4 个 GFP 蛋白的氨基酸序列（wild-type 序列），后续将基于这些序列设计突变
EXCLUSION_FILE = os.path.join('Exclusion_List.csv')  
# 不允许的序列清单，可能是失败序列、毒性序列等（用于过滤）


# --- 模型与生成参数 ---
ESM_MODEL_NAME = "esm2_t12_35M_UR50D" # 选择一个中等大小的ESM模型，平衡速度和性能
MAX_MUTATIONS = 6 # 比赛规则：最多6个突变
N_CANDIDATES_TO_GENERATE = 500 # 生成候选序列的数量（可调整）
TOP_N_SELECT = 10 # 最终选择的序列数量

# 检查是否有可用的 GPU
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# 设置随机种子以便结果可复现（可选）
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

/home/ma-user/anaconda3/envs/PyTorch-1.8/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


---

 # **第二步：获取原料并清洗！(数据加载与预处理)**

实验材料来了！我们需要处理好这些“原料”才能进行下一步。我们会：

1.  **加载“配方表” :** 读取 `GFP_data.xlsx` 文件，这里面记录了很多已知的 GFP 突变以及它们的亮度信息。这是我们学习的基础！
2.  **找到“原始模板” :** 从 `AAseqs of 4 GFP proteins.txt` 文件里找到我们这次的目标——avGFP 的野生型氨基酸序列。这是我们进行改造的基础。
3.  **拿到“禁止名单” :** 加载 `Exclusion_List.csv` 文件。记住！这里面的序列是**不能**提交的，所以我们得时刻对照着它。
4.  **筛选与转换 :** 我们只关心 avGFP 的数据，所以会先筛选一下。然后，最关键的一步：把 "G101A" 这种突变描述，应用到原始模板上，生成每一个突变体**完整**的氨基酸序列。这就像按配方修改原始模板，得到具体的成品序列。
5.  **清洗整理 :** 检查一下，确保生成的序列是有效的，亮度值也是正常的数字。把一些有问题的“原料”剔除掉，保证我们后续使用的是高质量数据。

处理完这些，我们就有了干净、规范的训练数据啦！

## 2.1 加载训练数据

In [8]:
print("Loading training data...")
try:
    gfp_df = pd.ExcelFile(TRAIN_DATA_FILE, sheet_name='brightness') # 假设亮度数据在名为 'brightness' 的 sheet
    print(f"Loaded {len(gfp_df)} rows from {TRAIN_DATA_FILE}")
except FileNotFoundError:
    print(f"Error: Training data file not found at {TRAIN_DATA_FILE}")
    # exit() # 如果文件不存在，可能需要停止执行

## 2.2 加载 avGFP 野生型序列 

In [9]:
print("Loading avGFP WT sequence...")
avGFP_WT_sequence = None
try:
    with open(WT_SEQ_FILE, 'r') as f:
        # 假设文件格式是 >Header \n Sequence \n >Header2...
        # 我们需要找到 avGFP 的序列
        header = ""
        seq_lines = []
        for line in f:
            if line.startswith('>'):
                # 如果找到了上一个序列，并且是avGFP，保存它
                if "avGFP" in header and seq_lines:
                    avGFP_WT_sequence = "".join(seq_lines).strip()
                    break # 找到后退出循环
                # 开始新的序列记录
                header = line.strip()
                seq_lines = []
            else:
                seq_lines.append(line.strip())
        # 处理文件最后一个序列的情况
        if avGFP_WT_sequence is None and "avGFP" in header and seq_lines:
             avGFP_WT_sequence = "".join(seq_lines).strip()

    if avGFP_WT_sequence:
        print(f"Found avGFP WT sequence (Length: {len(avGFP_WT_sequence)}).")
        # print(avGFP_WT_sequence) # 可以取消注释查看序列
    else:
        print("Error: avGFP WT sequence not found in", WT_SEQ_FILE)
        # 手动设置一个默认值或停止执行
        # avGFP_WT_sequence = "MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK" # 示例
        # print("Using default WT sequence.")
        # exit()

except FileNotFoundError:
    print(f"Error: WT sequence file not found at {WT_SEQ_FILE}")
    # exit()

Loading avGFP WT sequence...
Found avGFP WT sequence (Length: 238).


In [8]:
# --- 2.3 加载排除列表 ---
print("Loading exclusion list...")
try:
    exclusion_df = pd.read_csv(EXCLUSION_FILE)
    # 假设排除序列在名为 'sequences-not-submit' 的列中
    exclusion_sequences = set(exclusion_df['sequences-not-submit'].astype(str))
    print(f"Loaded {len(exclusion_sequences)} sequences into exclusion list.")
except FileNotFoundError:
    print(f"Error: Exclusion list file not found at {EXCLUSION_FILE}")
    exclusion_sequences = set() # 如果文件不存在，创建一个空集合
    print("Warning: Proceeding without an exclusion list.")
except KeyError:
    print(f"Error: Column 'sequences-not-submit' not found in {EXCLUSION_FILE}")
    exclusion_sequences = set()
    print("Warning: Proceeding without an exclusion list.")


# --- 2.4 预处理训练数据 ---
print("Preprocessing training data...")
# 筛选 avGFP 数据
avGFP_train_df = gfp_df[gfp_df['GFP type'] == 'avGFP'].copy()
print(f"Filtered down to {len(avGFP_train_df)} avGFP entries.")

# 定义函数：根据突变字符串生成完整序列
def generate_mutated_sequence(mutation_str, wt_sequence):
    """
    根据突变描述字符串和野生型序列生成突变后的完整序列。
    mutation_str: e.g., "WT", "G101A", "A12B:C34D"
    wt_sequence: 野生型氨基酸序列字符串
    """
    if not isinstance(mutation_str, str) or not wt_sequence:
        return None
    if mutation_str.strip().upper() == 'WT':
        return wt_sequence

    sequence = list(wt_sequence)
    mutations = mutation_str.split(':') # 支持多个突变，以冒号分隔
    valid_mutation_count = 0
    try:
        for mut in mutations:
            match = re.match(r'([A-Z])(\d+)([A-Z*.])$', mut.strip(), re.IGNORECASE) # 匹配 G101A, T203*, V163.
            if match:
                original_aa, pos, new_aa = match.groups()
                pos = int(pos) - 1 # 转换为 0-based index

                # 检查位置是否有效
                if pos < 0 or pos >= len(sequence):
                    # print(f"Warning: Invalid position {pos+1} in mutation '{mut}' for sequence length {len(sequence)}. Skipping mutation.")
                    continue # 跳过无效位置的突变

                # 检查原始氨基酸是否匹配 (可选，但建议)
                if sequence[pos].upper() != original_aa.upper():
                    # print(f"Warning: Original AA mismatch at position {pos+1} in mutation '{mut}'. Expected {sequence[pos]}, got {original_aa}. Applying mutation anyway.")
                    pass # 允许不匹配，但打印警告

                # 处理特殊字符
                if new_aa == '*': # 终止密码子 - 通常不希望出现在中间
                    # print(f"Warning: Stop codon '*' mutation '{mut}' encountered. Treating as deletion or invalid sequence for this tutorial.")
                    # 对于亮度预测，终止密码子通常导致无功能蛋白，可以返回None或特殊标记
                    return None # 或者根据需要处理
                elif new_aa == '.': # 表示与原氨基酸相同 (无突变)
                    new_aa = sequence[pos] # 保持不变

                sequence[pos] = new_aa.upper()
                valid_mutation_count += 1
            else:
                # print(f"Warning: Could not parse mutation '{mut}'. Skipping.")
                pass # 跳过无法解析的突变格式
        # 如果没有成功应用任何突变（可能是格式问题），返回None
        # if valid_mutation_count == 0 and mutations:
        #     return None
        return "".join(sequence)
    except Exception as e:
        # print(f"Error processing mutation string '{mutation_str}': {e}")
        return None # 返回 None 表示序列生成失败

# 应用函数生成序列
avGFP_train_df['full_sequence'] = avGFP_train_df['aaMutations'].apply(
    lambda x: generate_mutated_sequence(x, avGFP_WT_sequence)
)

# 清理数据：移除序列生成失败或亮度无效的行
original_len = len(avGFP_train_df)
avGFP_train_df.dropna(subset=['full_sequence', 'Brightness'], inplace=True)
# 确保亮度是数值类型
avGFP_train_df['Brightness'] = pd.to_numeric(avGFP_train_df['Brightness'], errors='coerce')
avGFP_train_df.dropna(subset=['Brightness'], inplace=True)

print(f"Removed {original_len - len(avGFP_train_df)} rows due to invalid sequences or brightness.")
print(f"Final training set size: {len(avGFP_train_df)}")

# 查看处理后的数据
print("\nSample of processed training data:")
print(avGFP_train_df[['aaMutations', 'Brightness', 'full_sequence']].head())

Loading training data...


TypeError: __init__() got an unexpected keyword argument 'sheet_name'

---
🧠 **第三步：让机器读懂蛋白质语言！(特征工程 - ESM 嵌入)**

蛋白质序列就像一种特殊的语言，直接丢给机器学习模型，它可能看不懂。我们需要把它翻译成模型能理解的“数字语言”。这就是 ESM 大显身手的时候了！但是，考虑到大家的 CPU 可能比较“吃力”，我们做了特别优化：

1.  **请个“轻量级”大师 👨‍🏫:** 我们会加载一个**更小巧、更快速**的 ESM 模型版本 (`esm2_t6_8M_UR50D`)。它虽然参数少一点，但跑起来会快很多！
2.  **“抽样”学习 📉:** 我们不需要把训练数据里成千上万条序列全都扔给 ESM 处理（那样太慢了！）。我们会从中**随机抽取一部分**（比如 1 万条）作为代表来进行学习。这就像考试前划重点，能大大节省时间！⏳
3.  **生成“数字指纹” 🔢:** 对于抽样选出的每一条蛋白质序列，ESM 模型会阅读它，并输出一个固定长度的数字列表（向量），我们称之为“嵌入”(Embedding)。这个嵌入就像是这条蛋白质序列的“数字指纹”，包含了它的关键生物学信息。

有了这些“数字指纹”，我们的机器学习模型就能更好地理解蛋白质序列了！

In [5]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import torch
import esm  # fair-esm库
import os
import time  # 用于计时
import warnings
warnings.filterwarnings('ignore')  # 忽略一些不影响结果的警告

# --- 优化常量 ---
# 根据设备选择合适的ESM模型
ESM_MODEL_NAME_CPU = "esm2_t6_8M_UR50D"
ESM_MODEL_NAME_GPU = "esm2_t30_150M_UR50D" # 可以选用更大的模型，例如 "esm2_t33_650M_UR50D"，取决于GPU内存

# 根据设备选择合适的批次大小
CPU_BATCH_SIZE = 8
GPU_BATCH_SIZE = 16 # GPU通常可以处理更大的批次，可以根据GPU内存调整

# 用于嵌入的最大训练样本数（可按需调整）
# 如果数据集小于此值，则使用所有样本
MAX_TRAIN_SAMPLES_FOR_EMBEDDING = 5000
SEED = 42 # 确保采样可复现

# --- 1. 设备检测 ---
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    ESM_MODEL_NAME = ESM_MODEL_NAME_GPU
    BATCH_SIZE = GPU_BATCH_SIZE
    print("检测到CUDA GPU，将使用GPU进行计算。")
else:
    DEVICE = torch.device("cpu")
    ESM_MODEL_NAME = ESM_MODEL_NAME_CPU
    BATCH_SIZE = CPU_BATCH_SIZE
    print("未检测到CUDA GPU或CUDA不可用，将使用CPU进行计算。")

print(f"正在使用设备: {DEVICE}")
print(f"将加载的ESM模型: {ESM_MODEL_NAME}")
print(f"将使用的批次大小: {BATCH_SIZE}")


# --- 3.0（优化）必要时对训练数据进行采样 ---
print("\n正在检查训练数据大小以进行采样...")
# 直接使用已加载的 avGFP_train_df
if len(avGFP_train_df) > MAX_TRAIN_SAMPLES_FOR_EMBEDDING:
    print(f"训练数据大小 ({len(avGFP_train_df)}) 超过限制 ({MAX_TRAIN_SAMPLES_FOR_EMBEDDING})。正在采样...")
    # 对DataFrame进行采样以减少用于嵌入的序列数量
    sampled_train_df = avGFP_train_df.sample(n=MAX_TRAIN_SAMPLES_FOR_EMBEDDING, random_state=SEED)
    print(f"使用 {len(sampled_train_df)} 个采样序列进行嵌入。")
else:
    print(f"训练数据大小 ({len(avGFP_train_df)}) 在限制范围内。使用所有序列。")
    # 使用完整的DataFrame
    sampled_train_df = avGFP_train_df.copy()  # 使用副本以避免修改原始数据

# --- 3.1 加载适合选定设备的ESM模型 ---
print(f"\n正在加载ESM模型: {ESM_MODEL_NAME} 到设备 {DEVICE}...")
start_time = time.time()
try:
    # 加载模型和字母表
    esm_model, alphabet = esm.pretrained.load_model_and_alphabet(ESM_MODEL_NAME)
    esm_model.eval()  # 设置为评估模式
    esm_model = esm_model.to(DEVICE)  # 将模型移动到选定的设备 (GPU or CPU)
    batch_converter = alphabet.get_batch_converter()
    print(f"ESM模型在 {time.time() - start_time:.2f} 秒内成功加载。")
except Exception as e:
    print(f"加载ESM模型时出错: {e}")
    print("请确保已安装 'fair-esm' 库 (pip install fair-esm) 且模型名称正确。")
    print("对于GPU使用，请确保已安装与CUDA兼容的PyTorch版本。")
    exit()  # 无法加载模型则退出

# --- 3.2 定义嵌入函数（适用于CPU和GPU） ---
def get_esm_embeddings(sequences, model, alphabet, batch_converter, device, batch_size):
    """
    在指定设备(CPU或GPU)上生成ESM嵌入（平均池化）。
    将输入数据和模型都移动到 `device`。
    """
    embeddings = []
    num_sequences = len(sequences)
    num_batches = (num_sequences + batch_size - 1) // batch_size
    model.eval() # 确保模型在评估模式
    model = model.to(device) # 确保模型在目标设备

    print(f"正在为 {num_sequences} 个序列生成嵌入，共 {num_batches} 个批次（批次大小: {batch_size}，设备: {device}）...")
    start_time_embed = time.time()

    with torch.no_grad():  # 对推理速度和内存至关重要
        for i in range(0, num_sequences, batch_size):
            batch_seqs = sequences[i:i + batch_size]
            batch_labels = [f"seq_{j + i}" for j in range(len(batch_seqs))]  # 每个批次项的唯一标签
            data = list(zip(batch_labels, batch_seqs))

            try:
                # 1. 准备批次
                _, _, batch_tokens = batch_converter(data)
                # 将令牌移动到目标设备 (GPU or CPU)
                batch_tokens = batch_tokens.to(device)

                # 2. 获取表示（只需要最后一层）
                # results = model(batch_tokens, repr_layers=[model.num_layers], return_contacts=False) # 旧版写法
                # 适配新版 fair-esm (>=2.0.0)
                results = model(batch_tokens, repr_layers=[model.num_layers])
                token_representations = results["representations"][model.num_layers]

                # 3. 对序列长度进行平均池化（忽略CLS/EOS/PAD令牌）
                # token_representations形状: [batch_size, seq_len+2, embed_dim]
                seq_repr_list = []
                for j, seq in enumerate(batch_seqs):
                    # +1 是因为 batch_converter 添加了 <cls> token
                    actual_len = len(seq)
                    # 从索引 1 开始到 actual_len 结束 (不包括 <eos> 和 padding)
                    # 注意：需要确保batch_converter没有添加其他的特殊token影响长度计算
                    # 对于标准esm模型，通常是<cls>...seq...<eos><pad>...
                    # 所以 token_representations[j, 1 : actual_len + 1, :] 是正确的
                    seq_tokens_repr = token_representations[j, 1 : actual_len + 1, :]
                    seq_repr = seq_tokens_repr.mean(dim=0) # 对实际序列长度的表示进行平均
                    seq_repr_list.append(seq_repr)

                batch_seq_repr = torch.stack(seq_repr_list, dim=0) # [batch_size, embed_dim]

                # 4. 存储结果 (移回CPU以聚合和后续处理，如转Numpy)
                embeddings.append(batch_seq_repr.cpu())

                if (i // batch_size + 1) % 10 == 0 or (i // batch_size + 1) == num_batches:  # 每10个批次或最后一个批次打印进度
                    elapsed_time = time.time() - start_time_embed
                    print(f"  已处理批次 {i // batch_size + 1}/{num_batches}... (耗时: {elapsed_time:.2f} 秒)")

            except RuntimeError as e:
                if "CUDA out of memory" in str(e) and device == torch.device("cuda"):
                    print(f"\n处理批次 {i // batch_size + 1} 时发生CUDA内存不足错误: {e}")
                    print(f"当前批次大小: {batch_size}。尝试减小批次大小或使用更小的模型。")
                    # 记录错误并跳过，但会导致数据丢失
                    embed_dim = model.embed_dim
                    error_placeholder = torch.full((len(batch_seqs), embed_dim), float('nan'), device='cpu') # 存放在CPU
                    embeddings.append(error_placeholder)
                    torch.cuda.empty_cache() # 尝试释放一些内存
                else:
                    print(f"处理批次 {i // batch_size + 1} 时发生运行时错误: {e}")
                    embed_dim = model.embed_dim
                    error_placeholder = torch.full((len(batch_seqs), embed_dim), float('nan'), device='cpu')
                    embeddings.append(error_placeholder)
            except Exception as e:
                print(f"处理批次 {i // batch_size + 1} 时发生未知错误: {e}")
                # 处理错误，例如跳过批次或用NaN填充
                embed_dim = model.embed_dim  # 获取预期维度
                error_placeholder = torch.full((len(batch_seqs), embed_dim), float('nan'), device='cpu')
                embeddings.append(error_placeholder)


    total_embed_time = time.time() - start_time_embed
    print(f"嵌入生成在 {total_embed_time:.2f} 秒内完成。")
    if num_sequences > 0:
      print(f"平均每个序列耗时: {total_embed_time / num_sequences:.4f} 秒")


    if not embeddings:
        return torch.tensor([])  # 返回空张量

    # 连接所有批次的结果
    try:
        full_embeddings = torch.cat(embeddings, dim=0)
    except RuntimeError as e:
        print(f"连接嵌入批次时出错: {e}")
        print("这可能发生在批次处理出错导致维度不匹配时。请检查之前的错误信息。")
        # 尝试找出有效批次并连接，或者返回错误
        try:
            embed_dim = model.embed_dim # 获取模型维度
            valid_embeddings = [emb for emb in embeddings if isinstance(emb, torch.Tensor) and emb.ndim == 2 and emb.shape[1] == embed_dim and not torch.isnan(emb).all()]
            if valid_embeddings:
                print("尝试仅连接有效的嵌入批次...")
                full_embeddings = torch.cat(valid_embeddings, dim=0)
            else:
                print("没有有效的嵌入批次可以连接。")
                return torch.tensor([]) # 返回空张量
        except Exception as concat_err:
             print(f"尝试连接有效嵌入时再次出错: {concat_err}")
             return torch.tensor([]) # 返回空张量


    return full_embeddings  # 作为单个张量返回 (在 CPU 上)

# --- 3.3 为（可能采样的）训练数据生成嵌入 ---
train_sequences_to_embed = sampled_train_df['full_sequence'].tolist()

X = None # 初始化 X
y = None # 初始化 y

if train_sequences_to_embed:
    # 获取嵌入作为PyTorch张量
    train_embeddings_tensor = get_esm_embeddings(
        train_sequences_to_embed,
        esm_model,
        alphabet,
        batch_converter,
        DEVICE,            # 传递检测到的设备
        batch_size=BATCH_SIZE # 传递适合设备的批次大小
    )

    if train_embeddings_tensor.numel() > 0: # 检查张量是否为空
        print(f"生成的嵌入张量形状: {train_embeddings_tensor.shape}")

        # 将张量转换为numpy数组以与scikit-learn兼容
        # 因为上面 append 时已经 .cpu()，所以这里 tensor 已经在 CPU 上
        train_embeddings_np = train_embeddings_tensor.numpy()

        # --- 处理嵌入过程中可能出现的NaN值 ---
        nan_rows_mask = np.isnan(train_embeddings_np).any(axis=1)
        if np.any(nan_rows_mask):
            num_nan_rows = nan_rows_mask.sum()
            print(f"警告: 在嵌入中发现 {num_nan_rows} 行NaN值（可能是由于错误）。正在移除这些行及其对应的原始数据。")

            # 过滤嵌入数组和相应的DataFrame行
            valid_indices_bool = ~nan_rows_mask
            # 获取原始sampled_train_df中对应valid_indices_bool为True的索引
            original_indices = sampled_train_df.index[valid_indices_bool]

            train_embeddings_np = train_embeddings_np[valid_indices_bool]
            # 使用 .loc 和原始索引进行过滤，确保正确对齐
            sampled_train_df_filtered = sampled_train_df.loc[original_indices]

            print(f"过滤后的嵌入数据大小: {train_embeddings_np.shape[0]}")
            print(f"过滤后的DataFrame大小: {len(sampled_train_df_filtered)}")


            # 检查过滤后是否还有数据
            if train_embeddings_np.shape[0] == 0:
                 print("\n错误: 移除NaN值后没有剩余数据。无法继续。")
                 X, y = None, None
            else:
                 # --- 为模型训练准备最终的X和y ---
                 if train_embeddings_np.shape[0] == len(sampled_train_df_filtered):
                     X = train_embeddings_np
                     y = sampled_train_df_filtered['Brightness'].values
                     print(f"\n准备好的X（嵌入）形状: {X.shape}, y（亮度）形状: {y.shape}")
                     print("步骤3（嵌入生成）完成。")
                 else:
                     # 这个情况理论上不应发生，因为我们同时过滤了两者
                     print(f"\n错误: NaN过滤后最终嵌入 ({train_embeddings_np.shape[0]}) 和DataFrame行 ({len(sampled_train_df_filtered)}) 数量不匹配。")
                     print("无法进行训练。请检查过滤逻辑。")
                     X, y = None, None

        else:
             # 没有NaN值，直接使用
             print("嵌入中未检测到NaN值。")
             X = train_embeddings_np
             # 直接从 sampled_train_df 获取 y，因为没有过滤
             y = sampled_train_df['Brightness'].values
             print(f"\n准备好的X（嵌入）形状: {X.shape}, y（亮度）形状: {y.shape}")
             print("步骤3（嵌入生成）完成。")

    else:
        print("\n嵌入过程未能生成有效的嵌入张量（可能所有批次都出错或输入为空）。")
        X, y = None, None

else:
    print("\n采样/预处理后没有可用的序列进行嵌入。")
    X, y = None, None

# --- 清理（可选，有助于释放内存，特别是GPU内存） ---
print("\n正在清理内存...")
del esm_model, alphabet, batch_converter
# 删除可能已创建的张量和numpy数组
if 'train_embeddings_tensor' in locals() and isinstance(train_embeddings_tensor, torch.Tensor):
    del train_embeddings_tensor
if 'train_embeddings_np' in locals():
    del train_embeddings_np
# 删除采样或过滤后的DataFrame副本
if 'sampled_train_df' in locals():
    del sampled_train_df
if 'sampled_train_df_filtered' in locals():
     del sampled_train_df_filtered

# 如果使用了GPU，显式清空缓存
if DEVICE == torch.device("cuda"):
    print("清空CUDA缓存...")
    torch.cuda.empty_cache()
print("清理完成。")

# --- 现在X和y（如果成功创建）已准备好用于步骤4（模型训练） ---
if X is not None and y is not None:
     print(f"\n数据准备就绪，可以进行模型训练。X shape: {X.shape}, y shape: {y.shape}")
     # 这里可以接上你的模型训练代码，例如：
     # from sklearn.model_selection import train_test_split
     # from sklearn.ensemble import RandomForestRegressor
     # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)
     # model = RandomForestRegressor(n_estimators=100, random_state=SEED)
     # model.fit(X_train, y_train)
     # print("模型训练完成。")
     # score = model.score(X_test, y_test)
     # print(f"模型在测试集上的 R^2 分数: {score:.4f}")
else:
     print("\n数据准备失败，无法进行模型训练。请检查之前的日志输出。")

# 示例: 如果X不为None，则打印(X.shape, y.shape)，否则打印("X is None")
# print("\n--- 最终检查 ---")
# print((X.shape, y.shape) if X is not None else "X is None")

检测到CUDA GPU，将使用GPU进行计算。
正在使用设备: cuda
将加载的ESM模型: esm2_t30_150M_UR50D
将使用的批次大小: 16

正在检查训练数据大小以进行采样...
训练数据大小 (51715) 超过限制 (5000)。正在采样...
使用 5000 个采样序列进行嵌入。

正在加载ESM模型: esm2_t30_150M_UR50D 到设备 cuda...
ESM模型在 8.33 秒内成功加载。
正在为 5000 个序列生成嵌入，共 313 个批次（批次大小: 16，设备: cuda）...
  已处理批次 10/313... (耗时: 10.54 秒)
  已处理批次 20/313... (耗时: 17.25 秒)
  已处理批次 30/313... (耗时: 23.79 秒)
  已处理批次 40/313... (耗时: 29.90 秒)
  已处理批次 50/313... (耗时: 36.21 秒)
  已处理批次 60/313... (耗时: 42.74 秒)
  已处理批次 70/313... (耗时: 49.45 秒)
  已处理批次 80/313... (耗时: 56.13 秒)
  已处理批次 90/313... (耗时: 62.88 秒)
  已处理批次 100/313... (耗时: 69.62 秒)
  已处理批次 110/313... (耗时: 76.36 秒)
  已处理批次 120/313... (耗时: 83.09 秒)
  已处理批次 130/313... (耗时: 89.87 秒)
  已处理批次 140/313... (耗时: 96.67 秒)
  已处理批次 150/313... (耗时: 103.48 秒)
  已处理批次 160/313... (耗时: 110.28 秒)
  已处理批次 170/313... (耗时: 117.07 秒)
  已处理批次 180/313... (耗时: 123.84 秒)
  已处理批次 190/313... (耗时: 130.64 秒)
  已处理批次 200/313... (耗时: 137.45 秒)
  已处理批次 210/313... (耗时: 144.25 秒)
  已处理批次 220/313... (耗时: 151.05 秒)
  已处理批次

---

🎓 **第四步：训练我们的亮度预测器！(模型训练 - 随机森林)**

现在，我们有了蛋白质的“数字指纹”（X，来自 ESM 嵌入）和它们对应的已知“亮度分数”（y）。是时候训练一个模型，让它学会根据指纹预测亮度了！

1.  **分班考试 📝:** 我们会把数据分成两部分：大部分（训练集）用来“教”模型学习规律，一小部分（验证集）用来“测试”模型学得怎么样，防止它“死记硬背”。
2.  **请“随机森林”老师 🌳🌳🌳:** 我们选择“随机森林” (Random Forest) 作为我们的预测模型。你可以把它想象成很多棵决策树组成的“智囊团”，它们各自学习，然后投票决定最终的预测结果。这种方法通常很稳健，效果也不错。
3.  **开始学习 🧑‍💻:** 用训练集的数据，“喂”给随机森林模型，让它努力学习从 ESM 指纹到亮度值的映射关系。
4.  **模拟测验 ✅:** 训练完成后，用我们之前留出的验证集来考考模型，看看它的预测准确度（比如用 R² 分数评估）怎么样。这能帮我们了解这个“亮度预测器”靠不靠谱。

训练完成后，我们就得到了一个可以预测未知序列亮度的模型啦！

In [6]:
# --- 4.1 划分训练集和验证集 ---
# 如果数据量较少，可以考虑交叉验证，这里用简单的划分
if len(X) > 10: # 确保有足够数据划分
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=SEED)
    print(f"Split data into training ({len(X_train)}) and validation ({len(X_val)}) sets.")
else:
    print("Dataset too small for validation split, using all data for training.")
    X_train, y_train = X, y
    X_val, y_val = None, None # 没有验证集

# --- 4.2 初始化并训练随机森林模型 ---
print("\nTraining Random Forest Regressor...")
rf_model = RandomForestRegressor(
    n_estimators=100, # 树的数量，可以调整
    random_state=SEED,
    n_jobs=-1, # 使用所有可用的 CPU 核心
    max_depth=20, # 限制树的深度，防止过拟合 (可调整)
    min_samples_leaf=3 # 叶节点最小样本数 (可调整)
)

rf_model.fit(X_train, y_train)
print("Random Forest training complete.")

# --- 4.3 (可选) 评估模型性能 ---
if X_val is not None:
    y_pred_val = rf_model.predict(X_val)
    r2 = r2_score(y_val, y_pred_val)
    print(f"\nModel Performance on Validation Set:")
    print(f"  R-squared (R²): {r2:.4f}")
    # R² 接近 1 表示模型拟合得较好，接近 0 或负数表示拟合很差
else:
    # 可以在训练集上评估，但这通常会过于乐观
    y_pred_train = rf_model.predict(X_train)
    r2_train = r2_score(y_train, y_pred_train)
    print("\nModel Performance on Training Set (may be optimistic):")
    print(f"  R-squared (R²): {r2_train:.4f}")

Split data into training (4000) and validation (1000) sets.

Training Random Forest Regressor...
Random Forest training complete.

Model Performance on Validation Set:
  R-squared (R²): 0.2793


---

💡 **第五步：设计新的潜力序列！(候选序列生成 - 知识引导)**

激动人心的创造环节来了！我们要设计全新的 avGFP 序列，目标是找到比原始模板更亮的！但我们不瞎猜，而是采取“知识引导”策略：

1.  **划定“重点区域” 🎯:** 这一步需要大家发挥聪明才智啦！你需要去查阅文献、分析我们提供的数据（尤其是 `beforetop10` 里的高分序列），甚至看看蛋白质的 3D 结构 (PDB 文件)，找出那些**最有可能影响亮度**的氨基酸位置。把这些位置汇总起来，形成一个“候选位点池”。这就像寻宝前先研究藏宝图！🗺️
2.  **组合突变 🧪:** 程序会从你精心挑选的“候选位点池”中，随机选择 1 到 6 个不同的位置。
3.  **引入变化 ✨:** 在选中的这几个位置上，随机地将原来的氨基酸替换成其他 19 种氨基酸中的一种。
4.  **大量制造 🏭:** 重复步骤 2 和 3 很多很多次（比如几千次），生成大量全新的、带有 1-6 个突变的 avGFP 候选序列。这些序列都是基于你的“知识引导”产生的，希望能更有潜力！

这样，我们就得到了一大批等待评估的新设计！

In [12]:
# --- 5.1 定义候选位点池 (示例) ---
# !!! 关键步骤：这个列表应该基于您的研究 !!!
# 示例：包含一些文献中提到的稳定/亮度相关位点，以及靠近发色团的位点

candidate_position_pool = [
    # 靠近发色团 (at 65-67)
    64, 68, 69, 70, 71, 72,
    # 文献中提到的与稳定/亮度相关的 (示例)
    10, 30, 64, # F64L 是 Superfolder GFP 的关键突变之一
    101, 105, 109,
    145, 147, 153, # M153T 也是 Superfolder GFP 突变
    163, 167, # V163A 也是 Superfolder GFP 突变
    171, 187,
    203, 205, 221, 231, 232, 235
]
# 转换为 0-based index 用于代码处理
candidate_position_pool_0based = [p - 1 for p in candidate_position_pool]
print(f"\nUsing a candidate position pool of {len(candidate_position_pool)} sites (0-based index):")
# print(candidate_position_pool_0based)

# --- 5.2 定义生成候选序列的函数 ---
amino_acids = 'ACDEFGHIKLMNPQRSTVWY' # 20种标准氨基酸

def generate_single_candidate(wt_sequence, position_pool_0based, max_mutations):
    """生成一个随机组合突变的候选序列"""
    num_mutations = random.randint(1, max_mutations)
    # 从池中随机选择 num_mutations 个不同的位置
    positions_to_mutate = random.sample(position_pool_0based, num_mutations)

    mutated_sequence = list(wt_sequence)
    mutation_details = []

    for pos in positions_to_mutate:
        original_aa = wt_sequence[pos]
        # 随机选择一个不同于原始氨基酸的新氨基酸
        possible_new_aas = [aa for aa in amino_acids if aa != original_aa]
        new_aa = random.choice(possible_new_aas)
        mutated_sequence[pos] = new_aa
        mutation_details.append(f"{original_aa}{pos+1}{new_aa}") # 记录突变 (1-based)

    return "".join(mutated_sequence), ":".join(sorted(mutation_details, key=lambda x: int(re.search(r'\d+', x).group()))) # 按位置排序突变描述

# --- 5.3 生成大量候选序列 ---
print(f"\nGenerating {N_CANDIDATES_TO_GENERATE} candidate sequences...")
candidate_sequences = {} # 使用字典存储 {sequence: mutation_str} 以确保唯一性
generated_count = 0
attempts = 0
max_attempts = N_CANDIDATES_TO_GENERATE * 5 # 设置尝试上限，防止无限循环

while generated_count < N_CANDIDATES_TO_GENERATE and attempts < max_attempts:
    attempts += 1
    seq, mut_str = generate_single_candidate(avGFP_WT_sequence, candidate_position_pool_0based, MAX_MUTATIONS)
    if seq not in candidate_sequences and seq != avGFP_WT_sequence: # 确保不重复且不是野生型
        candidate_sequences[seq] = mut_str
        generated_count += 1
        if generated_count % (N_CANDIDATES_TO_GENERATE // 10) == 0:
            print(f"  Generated {generated_count}/{N_CANDIDATES_TO_GENERATE} unique candidates...")

if generated_count < N_CANDIDATES_TO_GENERATE:
    print(f"Warning: Could only generate {generated_count} unique candidates after {attempts} attempts.")

candidate_list = list(candidate_sequences.keys())
mutation_list = [candidate_sequences[seq] for seq in candidate_list]
print(f"Generated a total of {len(candidate_list)} unique candidate sequences.")


Using a candidate position pool of 25 sites (0-based index):

Generating 500 candidate sequences...
  Generated 50/500 unique candidates...
  Generated 100/500 unique candidates...
  Generated 150/500 unique candidates...
  Generated 200/500 unique candidates...
  Generated 250/500 unique candidates...
  Generated 300/500 unique candidates...
  Generated 350/500 unique candidates...
  Generated 400/500 unique candidates...
  Generated 450/500 unique candidates...
  Generated 500/500 unique candidates...
Generated a total of 500 unique candidate sequences.


---

🔍 **第六步：预测、筛选与淘汰！(预测与过滤)**

我们创造了一堆新序列，现在要用之前训练好的“亮度预测器”来给它们打分，并进行严格筛选：

1.  **再次“翻译” 🗣️:** 对所有新生成的候选序列，再次使用**相同**的 ESM 模型（那个轻量级的）来计算它们的“数字指纹”（嵌入）。
2.  **预测亮度 🔮:** 把这些新序列的指纹输入到我们训练好的随机森林模型中，让模型预测出每一条新序列的亮度值。
3.  **对照“黑名单” 🚫:** 这是非常关键的一步！拿出 `Exclusion_List.csv`，检查我们预测出来的高分序列，**绝对不能**出现在这个名单里！如果在名单上，即使预测分数再高，也必须淘汰掉。
4.  **排序选优 🏆:** 将所有通过“黑名单”检查的候选序列，按照预测的亮度值从高到低排序。

经过这一轮，剩下的就是我们认为最有潜力、并且符合比赛规则的候选序列了！

In [13]:
import time
import torch
import esm # 假设 esm 库已导入
import numpy as np
import pandas as pd
# 假设 rf_model, candidate_list, mutation_list, exclusion_sequences,
# TOP_N_SELECT, CPU_BATCH_SIZE, get_esm_embeddings (来自上一步) 已经定义好
# 并且 avGFP_WT_sequence, candidate_position_pool_0based, MAX_MUTATIONS, N_CANDIDATES_TO_GENERATE 也已定义

# --- 修正：定义用于预测的 ESM 模型名称 ---
# !!! 关键：这里必须使用与训练 rf_model 时相同的 ESM 模型 !!!
# 根据之前的日志，rf_model 是用 640 维嵌入训练的 (来自 esm2_t30_150M_UR50D)
PREDICTION_ESM_MODEL_NAME = "esm2_t30_150M_UR50D" # <--- 确认这个模型与训练时一致

print(f"尝试加载用于预测的 ESM 模型: {PREDICTION_ESM_MODEL_NAME}")
start_time = time.time()
try:
    # 加载指定用于预测的 ESM 模型和字母表
    esm_model_pred, alphabet_pred = esm.pretrained.load_model_and_alphabet(PREDICTION_ESM_MODEL_NAME)
    batch_converter_pred = alphabet_pred.get_batch_converter()
    # 重新确定设备 (优先使用 GPU)
    DEVICE_pred = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    esm_model_pred.to(DEVICE_pred)
    esm_model_pred.eval() # 设置模型为评估模式 (关闭 dropout 等)
    print(f"用于预测的 ESM 模型 '{PREDICTION_ESM_MODEL_NAME}' 已加载到 {DEVICE_pred}，耗时 {time.time() - start_time:.2f} 秒。")
except Exception as e:
    print(f"加载 ESM 模型 {PREDICTION_ESM_MODEL_NAME} 时出错: {e}")
    print("请确保 'fair-esm' 已安装，模型名称正确，并且有足够的内存。")
    # 根据需要处理错误，例如退出
    exit()


# --- 6.1 为候选序列生成 ESM 嵌入 ---
print("\n使用预测模型为候选序列生成 ESM 嵌入...")
candidate_embeddings_np = np.array([]) # 初始化为空 numpy 数组

if candidate_list:
    # 确定批次大小 (如果使用GPU，可以尝试更大的批次)
    prediction_batch_size = CPU_BATCH_SIZE # 默认使用 CPU 批次大小 (来自之前的定义)
    if DEVICE_pred == torch.device("cuda"):
        # 如果是GPU，可以尝试更大的批次，例如 8, 16 或 32，取决于GPU内存和模型大小
        # 对于较大的模型如 150M，可能需要较小的批次大小
        prediction_batch_size = 8 # 示例值 (根据你的 GPU 内存调整，与训练时用的GPU_BATCH_SIZE可以不同)
        print(f"检测到 GPU，使用 GPU 批次大小: {prediction_batch_size}")
    else:
        print(f"使用 CPU 批次大小: {prediction_batch_size}")

    # *** 修复点：使用正确的函数名 'get_esm_embeddings' ***
    # 传递用于预测的模型、字母表、转换器和设备
    candidate_embeddings_tensor = get_esm_embeddings( # <-- 使用正确的函数名
        candidate_list,
        esm_model_pred,         # 使用预测模型
        alphabet_pred,        # 使用预测模型的字母表
        batch_converter_pred, # 使用预测模型的转换器
        DEVICE_pred,          # 使用预测模型所在的设备 (可能是 cuda 或 cpu)
        batch_size=prediction_batch_size # 使用调整后的批次大小
    )

    # 将嵌入结果转换为 NumPy 数组以用于 scikit-learn 模型
    # .cpu() 确保数据在 CPU 上，然后转换为 numpy
    candidate_embeddings_np = candidate_embeddings_tensor.cpu().numpy()

    print(f"候选序列嵌入的形状: {candidate_embeddings_np.shape}") # 检查维度是否正确 (应为 N x 640)

    # 检查是否有 NaN (如果嵌入过程中出错)
    if np.isnan(candidate_embeddings_np).any():
        print("警告：在候选序列嵌入中发现 NaN 值。将移除相应的候选序列。")
        nan_mask = np.isnan(candidate_embeddings_np).any(axis=1)
        # 需要同时过滤 candidate_list, mutation_list 和 embeddings
        # 使用列表推导式进行过滤
        original_count = len(candidate_list)
        candidate_list = [seq for i, seq in enumerate(candidate_list) if not nan_mask[i]]
        mutation_list = [mut for i, mut in enumerate(mutation_list) if not nan_mask[i]]
        candidate_embeddings_np = candidate_embeddings_np[~nan_mask]
        print(f"因 NaN 嵌入移除了 {original_count - len(candidate_list)} 个候选序列。")
        print(f"剩余候选序列数量: {len(candidate_list)}")
        print(f"过滤后的候选序列嵌入形状: {candidate_embeddings_np.shape}")

else:
    # candidate_embeddings_np 已经初始化为空数组
    print("上一步没有生成候选序列，跳过预测。")


# --- 6.2 预测亮度 ---
predicted_brightness = []
# 确保我们有有效的嵌入和候选列表来进行预测
# 并且嵌入的数量与候选列表的数量一致
if candidate_embeddings_np.shape[0] > 0 and len(candidate_list) == candidate_embeddings_np.shape[0]:
    print("\n正在为候选序列预测亮度...")
    try:
        # 使用加载的随机森林模型进行预测
        predicted_brightness = rf_model.predict(candidate_embeddings_np)
        print("预测完成。")
    except ValueError as ve: # 捕获更具体的 ValueError
        print(f"随机森林模型预测期间出错: {ve}")
        print("请确认用于预测的 ESM 模型生成的特征维度与训练 rf_model 时使用的维度一致。")
        # 如果预测失败，将结果列表清空，后续步骤会处理空结果
        predicted_brightness = []
    except Exception as e:
        print(f"随机森林模型预测期间发生未知错误: {e}")
        predicted_brightness = []
else:
    if not candidate_list:
         print("没有可用的候选序列进行预测。")
    elif candidate_embeddings_np.shape[0] == 0 and candidate_list:
         print("生成了候选序列，但未能生成有效的嵌入向量进行预测。")
    elif len(candidate_list) != candidate_embeddings_np.shape[0]:
         print("错误：经过 NaN 过滤后，候选序列数量与嵌入向量数量不匹配。")


# --- 6.3 组合结果并筛选 ---
final_candidates_formatted = pd.DataFrame() # 初始化为空 DataFrame

# 只有在成功生成预测并且数量与候选列表匹配时才继续
if len(candidate_list) > 0 and len(predicted_brightness) > 0 and len(candidate_list) == len(predicted_brightness):
    # 创建包含序列、突变和预测值的 DataFrame
    results_df = pd.DataFrame({
        'Sequence': candidate_list,
        'Mutations': mutation_list, # 确保 mutation_list 与 candidate_list 保持同步
        'PredictedBrightness': predicted_brightness
    })

    # 过滤掉排除列表中的序列
    print(f"\n根据排除列表 ({len(exclusion_sequences)} 个序列) 进行过滤...")
    initial_candidate_count = len(results_df)
    # 确保比较的是字符串类型
    results_df = results_df[~results_df['Sequence'].astype(str).isin(exclusion_sequences)]
    removed_count = initial_candidate_count - len(results_df)
    if removed_count > 0:
        print(f"移除了 {removed_count} 个在排除列表中的序列。")
    else:
        print("候选列表中的序列均不在排除列表中。")

    # 按预测亮度降序排序
    results_df = results_df.sort_values(by='PredictedBrightness', ascending=False)

    # 选择 Top N 个结果
    final_candidates = results_df.head(TOP_N_SELECT).copy() # 使用 .copy() 避免 SettingWithCopyWarning

    print(f"\n预测出的 Top {min(TOP_N_SELECT, len(final_candidates))} 个候选序列 (已排除):") # 显示实际选出的数量

    if not final_candidates.empty:
        # 为了更清晰地展示，可以添加一个 ID 列
        final_candidates.insert(0, 'Sequence ID', [f'Candidate_{i+1}' for i in range(len(final_candidates))])
        # 调整列顺序以符合提交格式要求
        final_candidates_formatted = final_candidates[['Sequence ID', 'Mutations', 'Sequence', 'PredictedBrightness']]
        # 使用 display 或 print 显示 DataFrame
        try:
            from IPython.display import display
            display(final_candidates_formatted) # 在 Jupyter 环境中友好显示
        except ImportError:
            print(final_candidates_formatted.to_string()) # 在非 IPython 环境中打印完整 DataFrame
    else:
        print("经过过滤和筛选后，没有剩余的候选序列。")

elif not candidate_list:
     print("\n没有生成候选序列或候选序列在预测前已被过滤掉。")
elif not predicted_brightness:
     print("\n预测步骤失败或没有产生结果。请检查之前的错误信息。")
else: # candidate_list 和 predicted_brightness 长度不匹配
     print("\n错误：最终候选序列数量与预测结果数量不匹配。无法继续处理。")

# --- 清理预测模型占用的内存 ---
print("\n清理预测模型内存...")
del esm_model_pred, alphabet_pred, batch_converter_pred
if 'candidate_embeddings_tensor' in locals():
    del candidate_embeddings_tensor
if 'candidate_embeddings_np' in locals():
    del candidate_embeddings_np
if DEVICE_pred == torch.device("cuda"):
    print("清空CUDA缓存...")
    torch.cuda.empty_cache()
print("预测模型清理完成。")

尝试加载用于预测的 ESM 模型: esm2_t30_150M_UR50D
用于预测的 ESM 模型 'esm2_t30_150M_UR50D' 已加载到 cuda，耗时 2.48 秒。

使用预测模型为候选序列生成 ESM 嵌入...
检测到 GPU，使用 GPU 批次大小: 8
正在为 500 个序列生成嵌入，共 63 个批次（批次大小: 8，设备: cuda）...
  已处理批次 10/63... (耗时: 2.23 秒)
  已处理批次 20/63... (耗时: 4.39 秒)
  已处理批次 30/63... (耗时: 7.31 秒)
  已处理批次 40/63... (耗时: 11.05 秒)
  已处理批次 50/63... (耗时: 14.43 秒)
  已处理批次 60/63... (耗时: 17.84 秒)
  已处理批次 63/63... (耗时: 18.71 秒)
嵌入生成在 18.71 秒内完成。
平均每个序列耗时: 0.0374 秒
候选序列嵌入的形状: (500, 640)

正在为候选序列预测亮度...
预测完成。

根据排除列表 (739 个序列) 进行过滤...
移除了 1 个在排除列表中的序列。

预测出的 Top 6 个候选序列 (已排除):


,Sequence ID,Mutations,Sequence,PredictedBrightness
328,Candidate_1,V163T,MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,3.514671
496,Candidate_2,E235T,MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,3.490361
479,Candidate_3,F71L,MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,3.485293
412,Candidate_4,V163D,MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,3.483591
179,Candidate_5,L221T,MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,3.482050
82,Candidate_6,E235S,MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,3.481949



清理预测模型内存...
清空CUDA缓存...
预测模型清理完成。


---
📄 **第七步：成果展示与未来展望！(输出与后续步骤)**

太棒了！我们终于走完了整个流程！🎉

1.  **最终名单 🏅:** 从上一轮筛选排序后的结果中，选出预测亮度最高的 **Top 6** 条序列。
2.  **整理提交 ✍️:** 将这 6 条序列的信息（比如给它们起个名字 `Sequence ID`、记录清楚具体的突变 `Mutations`、以及完整的序列 `Sequence`）整理成比赛要求的 `.csv` 文件格式。
3.  **下一步行动 🤔:** 这个教程只是一个起点！要在比赛中获得好成绩，你还需要：
    *   **深入研究 📚:** 花更多时间优化你的“候选位点池”，让它更精准！
    *   **平衡稳定性 🔥:** 这次我们主要关注了亮度，但比赛还要求**热稳定性**！你需要思考如何引入提高稳定性的策略（比如参考 Superfolder GFP 的突变，分析 PDB 结构寻找可以优化的点）。这可能是个多目标优化的挑战！
    *   **升级装备 🚀:** 可以尝试不同的机器学习模型、更仔细地调整模型参数，甚至探索更高级的技术。
    *   **打包文档 📦:** 别忘了按要求整理好你的代码 (`.zip`)，并写一份清晰的“设计思路”文档 (`.docx` 或 `.pdf`)，解释你是怎么做的。

In [ ]:
# --- 7.1 准备提交格式 (示例) ---
# 比赛要求提交 CSV，包含 'Sequence ID', 'Mutations', 'Full Sequence'
# 我们已经有了类似格式的 DataFrame 'final_candidates_formatted'

# 如果需要保存为 CSV 文件：
output_filename = "my_top_brightness_candidates.csv"
if not final_candidates_formatted.empty:
    # 选择需要的列
    submission_df = final_candidates_formatted[['Sequence ID', 'Mutations', 'Sequence']].copy()
    # 重命名列以完全匹配（如果需要）
    # submission_df.rename(columns={'Sequence': 'Full Sequence'}, inplace=True) # 假设需要 'Full Sequence' 列名
    submission_df.to_csv(output_filename, index=False)
    print(f"\nSuccessfully saved top {len(submission_df)} candidates to {output_filename}")
else:
    print("\nNo final candidates to save.")


# --- 7.2 后续步骤 --- 
print("\n--- 教程已完成 ---") 
print("后续可能的优化步骤：") 
print("1.  **优化位置池：** 目前使用的数据量和批次非常小，您可以进行全面的文献/数据/结构分析，以创建更好的`候选位置池`。") 
print("2.  **考虑稳定性：** 本教程仅关注亮度。你需要纳入预测/提高热稳定性的策略或模型（例如，使用PDB结构、已知的稳定突变）。这可能涉及多目标优化。") 
print("3.  **改进模型：** 尝试不同的ESM模型、回归算法（例如梯度提升）、超参数调整，或更先进的技术，如微调ESM。") 
print("4.  **代码打包：** 根据要求将代码整理成可运行的脚本或包（.zip）。包含一个README文件。") 
print("5.  **设计原理：** 撰写一份清晰的文档，解释你的方法、选择特定位置/突变的原因以及所使用的方法。") 
print("6.  **提交：** 准备最终的CSV文件，其中准确包含6条序列，确保它们不在排除列表中且符合突变限制。") 

---


以下演示了其它方案 使用Saprot模型用于向量嵌入

---
🧠 **(备选) 第三步 B：让 SaProt 读懂蛋白质语言！(特征工程 - SaProt 嵌入)**

除了 ESM，还有其他强大的蛋白质语言模型，例如 SaProt。SaProt 通常基于 Transformer 架构，并通过 Hugging Face 的 `transformers` 库加载。

与之前使用的轻量级 ESM 模型相比，SaProt 模型（如 `ECAS/SaProt_650M_AF2`）参数量可能更大，理论上可能捕捉更丰富的信息，但也需要更多的计算资源（内存和时间），尤其是在 CPU 上。

下面我们将展示如何加载 SaProt 并用它来生成嵌入。你可以选择性地用这些嵌入替换前面 ESM 生成的嵌入，用于后续的模型训练和评估。

**注意:** 下一步需要安装 `transformers` 库。

In [14]:
# 安装 Hugging Face Transformers 库
!pip install transformers sentencepiece

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [15]:
# -*- coding: utf-8 -*-
# --- SaProt 完整工作流程单元 ---
import os
import sys

# === 在导入 transformers 之前设置 Hugging Face 镜像 ===
# 检查环境变量是否已设置，如果未设置，则使用镜像
# 这样做可以最大程度确保 transformers 库在初始化时就使用镜像地址
hf_endpoint = os.environ.get('HF_ENDPOINT')
mirror_endpoint = 'https://hf-mirror.com' # 指定镜像地址

if hf_endpoint != mirror_endpoint: # 仅在未设置或设置不正确时进行设置
    print(f"HF_ENDPOINT 未设置或非预期值。将尝试设置 Hugging Face 端点为镜像: {mirror_endpoint}")
    os.environ['HF_ENDPOINT'] = mirror_endpoint
    # 验证是否设置成功 (可能需要重启内核/环境才能完全生效，但在脚本内尽力设置)
    current_endpoint = os.environ.get('HF_ENDPOINT')
    if current_endpoint == mirror_endpoint:
        print(f"当前脚本执行环境中 HF_ENDPOINT 已设置为: {current_endpoint}")
    else:
        print(f"警告: 尝试设置 HF_ENDPOINT，但读取值仍为 {current_endpoint}。环境变量可能未完全生效。")
        # 如果脚本内设置无效，可能需要在外部环境（如启动脚本或系统环境变量）中设置
else:
    print(f"检测到 HF_ENDPOINT 已设置为镜像: {hf_endpoint}，将继续使用此地址。")

# === 1. 导入库与基础设置 ===
import pandas as pd
import numpy as np
import torch
import time
import random
import re
import warnings
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
# 尝试导入 transformers (现在应该会使用设置的 HF_ENDPOINT)
try:
    from transformers import AutoTokenizer, AutoModel
except ImportError:
    print("错误: 无法导入 transformers 库。请确保已正确安装 (pip install transformers)。")
    sys.exit(1) # 无法导入则退出

# 在Jupyter Notebook中优化DataFrame的显示
try:
    from IPython.display import display
except ImportError:
    # 如果不在IPython环境中，定义一个简单的display函数
    def display(x):
        print(x)

warnings.filterwarnings('ignore') # 忽略不影响结果的警告

print("--- 开始 SaProt 工作流程 ---")

# === 2. 安装依赖 (注释掉，建议在环境级别管理) ===
# print("\n[步骤 1/9] 正在安装所需库 (transformers, sentencepiece)...")
# !pip install transformers sentencepiece -q # -q 表示静默安装
# print("库安装检查完成。")


# === 3. SaProt 常量设置与设备检测 ===

# --- 模型名称 (来自镜像) ---
SAPROT_MODEL_NAME = "westlake-repl/SaProt_35M_AF2"
print(f"指定使用的 SaProt 模型 (来自镜像): {SAPROT_MODEL_NAME}")

# --- 批次大小设置 ---
SAPROT_BATCH_SIZE_CPU = 4
SAPROT_BATCH_SIZE_GPU = 32 # 35M 模型通常允许较大的批次

# --- 设备检测 ---
if torch.cuda.is_available():
    DEVICE_SAPROT = torch.device("cuda")
    SAPROT_BATCH_SIZE = SAPROT_BATCH_SIZE_GPU
    print("检测到 CUDA GPU，将使用 GPU 进行 SaProt 计算。")
    try:
        gpu_mem_gb = torch.cuda.get_device_properties(0).total_memory / (1024**3)
        print(f"GPU 总显存: {gpu_mem_gb:.2f} GB")
    except Exception as e:
        print(f"无法获取GPU内存信息: {e}")
else:
    DEVICE_SAPROT = torch.device("cpu")
    SAPROT_BATCH_SIZE = SAPROT_BATCH_SIZE_CPU
    print("未检测到 CUDA GPU 或 CUDA 不可用，将使用 CPU 进行 SaProt 计算。")

# --- 最终确认打印 ---
print(f"最终使用的 SaProt 设备: {DEVICE_SAPROT}")
print(f"最终使用的 SaProt 模型: {SAPROT_MODEL_NAME}")
print(f"最终使用的 SaProt 批次大小: {SAPROT_BATCH_SIZE}")


# === 检查先前单元格定义的必要变量/函数是否存在 ===
# 确保运行此单元格前，已成功运行定义这些变量和函数的单元格
# !!! 注意: 请确保在运行此脚本/单元格之前，这些变量已经在您的环境中定义 !!!
required_vars = ['avGFP_train_df', 'avGFP_WT_sequence', 'exclusion_sequences',
                 'MAX_TRAIN_SAMPLES_FOR_EMBEDDING', 'SEED', 'candidate_position_pool_0based',
                 'amino_acids', 'MAX_MUTATIONS', 'N_CANDIDATES_TO_GENERATE', 'TOP_N_SELECT',
                 'generate_single_candidate']
missing = [var for var in required_vars if not (var in locals() or var in globals())]
if missing:
    print("\n错误: 未找到所有必需的变量或函数。")
    print(f"请确保已首先运行定义了以下内容的单元格: {', '.join(missing)}")
    raise NameError(f"缺少来自先前笔记本单元格的必需变量: {', '.join(missing)}")
else:
    print("\n[步骤 1/9] 检查依赖变量/函数... 通过。")


# === 4. 加载 SaProt Tokenizer 和模型 ===
# 现在调用 from_pretrained 时，应该使用脚本顶部设置的环境变量
print(f"\n[步骤 2/9] 正在加载 SaProt Tokenizer 和模型: {SAPROT_MODEL_NAME} 到 {DEVICE_SAPROT}...")
start_time = time.time()
try:
    # 直接调用，依赖于顶部的环境变量设置
    saprot_tokenizer = AutoTokenizer.from_pretrained(SAPROT_MODEL_NAME)
    saprot_model = AutoModel.from_pretrained(SAPROT_MODEL_NAME)

    # --- 优化: 尝试使用 torch.compile (保持不变) ---
    use_torch_compile = False
    if DEVICE_SAPROT == torch.device("cuda") and hasattr(torch, 'compile'):
        try:
            print("  尝试使用 torch.compile 优化模型 (可能需要一些时间)...")
            # 'reduce-overhead' 模式通常适用于推理
            saprot_model = torch.compile(saprot_model, mode="reduce-overhead", fullgraph=True)
            print("  torch.compile 应用成功！")
            use_torch_compile = True
        except Exception as compile_err:
            print(f"  torch.compile 失败: {compile_err}。将使用未优化的模型。")

    # --- 模型设置与设备转移 (保持不变) ---
    saprot_model.eval() # 设置为评估模式
    saprot_model = saprot_model.to(DEVICE_SAPROT) # 将模型参数移动到 GPU 或 CPU

    print(f"SaProt 模型加载并在 {DEVICE_SAPROT} 上准备就绪，耗时 {time.time() - start_time:.2f} 秒。")

except OSError as e:
    # 捕获 OSError，它通常包含连接或文件未找到的错误
    print(f"\n!!! 加载模型时发生 OSError: {e}")
    print("这通常意味着：")
    print("  1. 网络连接问题：无法访问 HF_ENDPOINT 指定的地址。")
    print(f"     - 检查镜像地址 '{os.environ.get('HF_ENDPOINT')}' 是否可访问。")
    print(f"     - 检查你的网络连接和防火墙设置。")
    print("  2. 模型名称错误或镜像上不存在该模型。")
    print(f"     - 确认模型 '{SAPROT_MODEL_NAME}' 在镜像站点上确实存在。")
    print("  3. 缓存问题：本地缓存可能已损坏或不完整。")
    print("     - 尝试清除 Hugging Face 缓存 (通常在 ~/.cache/huggingface/hub 或 ~/.cache/huggingface/transformers)。")
    print("  4. 环境变量未生效：如果看到错误仍然指向 huggingface.co，可能需要重启 Python 内核/环境或在外部设置环境变量。")
    # 可以选择停止执行
    raise e
except Exception as e:
    # 捕获其他可能的异常
    print(f"\n!!! 加载 SaProt 模型时发生未知错误: {e}")
    print(f"使用的模型标识符: {SAPROT_MODEL_NAME}")
    print(f"使用的镜像端点: {os.environ.get('HF_ENDPOINT')}")
    raise # 重新引发异常，停止执行


# === 5. 定义 SaProt 嵌入函数 ===
print("\n[步骤 3/9] 正在定义 SaProt 嵌入函数...")
def get_saprot_embeddings(sequences, model, tokenizer, device, batch_size, use_compile=False):
    """
    使用 SaProt 模型为序列列表生成嵌入 (采用平均池化)。
    针对 GPU 进行了优化，自动处理设备转移和批处理。
    """
    embeddings = [] # 用于存储每个批次的嵌入结果
    num_sequences = len(sequences)
    if num_sequences == 0:
        print("  输入序列列表为空，无需生成嵌入。")
        return torch.tensor([])

    num_batches = (num_sequences + batch_size - 1) // batch_size
    print(f"  准备为 {num_sequences} 个序列生成 SaProt 嵌入，共 {num_batches} 批 (批大小: {batch_size})...")
    start_time_embed = time.time()

    model.eval() # 再次确保是评估模式

    with torch.no_grad(): # 优化推理速度和内存
        for i in range(0, num_sequences, batch_size):
            batch_seqs = sequences[i:i + batch_size]
            batch_seqs_spaced = [" ".join(list(s)) for s in batch_seqs] # SaProt 需要空格分隔
            current_batch_num = i // batch_size + 1

            try:
                inputs = tokenizer(batch_seqs_spaced, add_special_tokens=True, padding=True, truncation=True, return_tensors='pt')
                inputs = {key: val.to(device) for key, val in inputs.items()} # 数据移动到 GPU/CPU
                outputs = model(**inputs) # 模型推理
                last_hidden_states = outputs.last_hidden_state

                # 平均池化 (考虑 attention mask)
                mask = inputs['attention_mask'].unsqueeze(-1).expand(last_hidden_states.size()).float()
                sum_hidden_states = torch.sum(last_hidden_states * mask, dim=1)
                sum_mask = torch.clamp(mask.sum(dim=1), min=1e-9)
                mean_pooled_embeddings = sum_hidden_states / sum_mask

                embeddings.append(mean_pooled_embeddings.cpu()) # 结果移回 CPU

                # 打印进度
                if current_batch_num % max(1, num_batches // 10) == 0 or current_batch_num == num_batches:
                     progress_percent = (current_batch_num / num_batches) * 100
                     elapsed = time.time() - start_time_embed
                     print(f"    已处理批次 {current_batch_num}/{num_batches} ({progress_percent:.1f}%) - 耗时: {elapsed:.2f} 秒")

            except RuntimeError as e:
                # 处理内存不足等运行时错误
                if "CUDA out of memory" in str(e) and device == torch.device("cuda"):
                    print(f"\n    处理批次 {current_batch_num} 时发生 CUDA 内存不足错误!")
                    print(f"    当前批次大小: {batch_size}。请尝试减小 'SAPROT_BATCH_SIZE_GPU'。")
                    torch.cuda.empty_cache()
                    hidden_size = model.config.hidden_size if hasattr(model, 'config') else 768 # 尝试获取维度
                    error_placeholder = torch.full((len(batch_seqs), hidden_size), float('nan'), device='cpu')
                    embeddings.append(error_placeholder)
                else:
                    print(f"    处理 SaProt 批次 {current_batch_num} 时发生运行时错误: {e}")
                    hidden_size = model.config.hidden_size if hasattr(model, 'config') else 768
                    error_placeholder = torch.full((len(batch_seqs), hidden_size), float('nan'), device='cpu')
                    embeddings.append(error_placeholder)
            except Exception as e:
                # 处理其他错误
                print(f"    处理 SaProt 批次 {current_batch_num} 时发生未知错误: {e}")
                hidden_size = model.config.hidden_size if hasattr(model, 'config') else 768
                error_placeholder = torch.full((len(batch_seqs), hidden_size), float('nan'), device='cpu')
                embeddings.append(error_placeholder)

    total_embed_time = time.time() - start_time_embed
    print(f"  SaProt 嵌入生成完成，总耗时: {total_embed_time:.2f} 秒。")
    if num_sequences > 0:
        print(f"  平均每个序列耗时: {total_embed_time / num_sequences:.4f} 秒。")

    # 拼接嵌入结果
    if not embeddings: return torch.tensor([])
    try:
        full_embeddings = torch.cat(embeddings, dim=0)
    except Exception as cat_err:
        print(f"  拼接嵌入批次时出错: {cat_err}。尝试拼接有效部分...")
        valid_embeddings = [emb for emb in embeddings if isinstance(emb, torch.Tensor) and emb.ndim == 2 and not torch.isnan(emb).all()]
        if valid_embeddings:
             try:
                 expected_dim = valid_embeddings[0].shape[1]
                 valid_embeddings = [emb for emb in valid_embeddings if emb.shape[1] == expected_dim]
                 if valid_embeddings:
                     full_embeddings = torch.cat(valid_embeddings, dim=0)
                 else: return torch.tensor([]) # 没有维度一致的
             except Exception: return torch.tensor([]) # 拼接有效部分也失败
        else: return torch.tensor([]) # 没有有效的

    return full_embeddings

print("SaProt 嵌入函数定义完成。")


# === 6. 为训练数据生成 SaProt 嵌入 ===
print(f"\n[步骤 4/9] 正在为训练数据生成 SaProt 嵌入...")
# (采样逻辑保持不变)
if len(avGFP_train_df) > MAX_TRAIN_SAMPLES_FOR_EMBEDDING:
    print(f"  训练数据量 ({len(avGFP_train_df)}) 较大，将采样 {MAX_TRAIN_SAMPLES_FOR_EMBEDDING} 条用于嵌入和训练。")
    sampled_train_df_saprot = avGFP_train_df.sample(n=MAX_TRAIN_SAMPLES_FOR_EMBEDDING, random_state=SEED)
else:
    print(f"  使用全部 {len(avGFP_train_df)} 条训练数据进行嵌入。")
    sampled_train_df_saprot = avGFP_train_df.copy()

train_sequences_saprot = sampled_train_df_saprot['full_sequence'].tolist()
X_saprot = None # 初始化为 None
y_saprot = None

if train_sequences_saprot:
    X_saprot_tensor = get_saprot_embeddings(
        train_sequences_saprot, saprot_model, saprot_tokenizer, DEVICE_SAPROT,
        batch_size=SAPROT_BATCH_SIZE, use_compile=use_torch_compile
    )
    if X_saprot_tensor.numel() > 0:
        print(f"  生成的训练数据嵌入张量形状: {X_saprot_tensor.shape}")
        X_saprot_np = X_saprot_tensor.numpy()
        # 处理 NaN
        nan_rows_mask_train = np.isnan(X_saprot_np).any(axis=1)
        if np.any(nan_rows_mask_train):
            num_nan_rows = nan_rows_mask_train.sum()
            print(f"  警告: 在训练嵌入中发现 {num_nan_rows} 行 NaN 值。将移除它们。")
            X_saprot_np = X_saprot_np[~nan_rows_mask_train]
            sampled_train_df_saprot = sampled_train_df_saprot[~nan_rows_mask_train]
            print(f"  移除 NaN 后，剩余训练数据: {len(sampled_train_df_saprot)} 条")

        if X_saprot_np.shape[0] > 0 and X_saprot_np.shape[0] == len(sampled_train_df_saprot):
            X_saprot = X_saprot_np
            y_saprot = sampled_train_df_saprot['Brightness'].values
            print(f"  准备好的 X_saprot (嵌入) 形状: {X_saprot.shape}, y_saprot (亮度) 形状: {y_saprot.shape}")
        else:
            print("  错误: 处理 NaN 或嵌入失败后，X 和 y 数据不匹配或为空。")
    else:
        print("  警告: 未能成功为训练数据生成 SaProt 嵌入。")
else:
    print("  没有找到用于生成嵌入的训练序列。")

if X_saprot is None or y_saprot is None:
    print("!!! 错误: 未能准备 SaProt 训练数据 (X_saprot 或 y_saprot 为空)。后续步骤可能失败。")


# === 7. 使用 SaProt 嵌入训练随机森林回归模型 ===
print("\n[步骤 5/9] 正在使用 SaProt 嵌入训练随机森林回归器...")
rf_model_saprot = None # 初始化模型变量
if X_saprot is not None and y_saprot is not None and X_saprot.shape[0] > 0:
    if len(X_saprot) > 10:
        X_train_saprot, X_val_saprot, y_train_saprot, y_val_saprot = train_test_split(
            X_saprot, y_saprot, test_size=0.2, random_state=SEED
        )
        print(f"  已将数据分割为训练集 ({len(X_train_saprot)}) 和验证集 ({len(X_val_saprot)})。")
    else:
        print("  数据集过小，无法进行验证分割，将使用所有数据进行训练。")
        X_train_saprot, y_train_saprot = X_saprot, y_saprot
        X_val_saprot, y_val_saprot = None, None

    rf_model_saprot = RandomForestRegressor(
        n_estimators=100, random_state=SEED, n_jobs=-1, max_depth=20,
        min_samples_leaf=3, oob_score=(X_val_saprot is None)
    )
    start_time = time.time()
    print("  开始训练随机森林模型...")
    rf_model_saprot.fit(X_train_saprot, y_train_saprot)
    print(f"  随机森林训练完成，耗时 {time.time() - start_time:.2f} 秒。")

    # 模型评估
    if X_val_saprot is not None and y_val_saprot is not None:
        y_pred_val_saprot = rf_model_saprot.predict(X_val_saprot)
        r2_saprot_val = r2_score(y_val_saprot, y_pred_val_saprot)
        print(f"  基于 SaProt 的模型在【验证集】上的 R² 分数: {r2_saprot_val:.4f}")
    elif hasattr(rf_model_saprot, 'oob_score_') and rf_model_saprot.oob_score_:
         print(f"  基于 SaProt 的模型袋外 (OOB) R² 分数: {rf_model_saprot.oob_score_:.4f}")
    else:
        y_pred_train_saprot = rf_model_saprot.predict(X_train_saprot)
        r2_saprot_train = r2_score(y_train_saprot, y_pred_train_saprot)
        print(f"  基于 SaProt 的模型在【训练集】上的 R² 分数: {r2_saprot_train:.4f} (注意: 可能偏高)")
else:
    print("  由于之前的步骤未能成功准备 X_saprot 和 y_saprot，跳过模型训练。")


# === 8. 生成候选序列 ===
print(f"\n[步骤 6/9] 正在生成 {N_CANDIDATES_TO_GENERATE} 个候选序列...")
candidate_sequences_saprot = {}
generated_count = 0
attempts = 0
max_attempts = N_CANDIDATES_TO_GENERATE * 10
start_time_gen = time.time()

# !!! 确保 generate_single_candidate 函数在此作用域可用 !!!
if 'generate_single_candidate' not in globals() and 'generate_single_candidate' not in locals():
     raise NameError("函数 'generate_single_candidate' 未定义。请确保它在之前的单元格中已运行。")

while generated_count < N_CANDIDATES_TO_GENERATE and attempts < max_attempts:
    attempts += 1
    try:
        seq, mut_str = generate_single_candidate(
            avGFP_WT_sequence, candidate_position_pool_0based, MAX_MUTATIONS
        )
        if seq not in candidate_sequences_saprot and seq != avGFP_WT_sequence:
            candidate_sequences_saprot[seq] = mut_str
            generated_count += 1
            if generated_count % max(1, N_CANDIDATES_TO_GENERATE // 10) == 0 or generated_count == N_CANDIDATES_TO_GENERATE :
                 print(f"  已生成 {generated_count}/{N_CANDIDATES_TO_GENERATE} 个唯一候选序列...")
    except Exception as gen_err:
        print(f"生成候选序列时出错 (尝试 {attempts}): {gen_err}")
        # 可以选择继续或停止
        # break

gen_duration = time.time() - start_time_gen
if generated_count < N_CANDIDATES_TO_GENERATE:
    print(f"  警告: 尝试 {attempts} 次后，仅生成了 {generated_count} 个独特的候选序列 (目标 {N_CANDIDATES_TO_GENERATE})。")
else:
    print(f"  成功生成 {generated_count} 个独特的候选序列。")
print(f"  候选序列生成耗时: {gen_duration:.2f} 秒。")

candidate_list_saprot = list(candidate_sequences_saprot.keys())
mutation_list_saprot = [candidate_sequences_saprot[seq] for seq in candidate_list_saprot]


# === 9. 为候选序列生成 SaProt 嵌入 ===
print("\n[步骤 7/9] 正在为候选序列生成 SaProt 嵌入...")
candidate_embeddings_saprot_np = np.array([]) # 初始化
if candidate_list_saprot:
    print(f"  将为 {len(candidate_list_saprot)} 个候选序列生成嵌入。")
    candidate_embeddings_saprot_tensor = get_saprot_embeddings(
        candidate_list_saprot, saprot_model, saprot_tokenizer, DEVICE_SAPROT,
        batch_size=SAPROT_BATCH_SIZE, use_compile=use_torch_compile
    )
    if candidate_embeddings_saprot_tensor.numel() > 0:
        print(f"  生成的候选序列嵌入张量形状: {candidate_embeddings_saprot_tensor.shape}")
        candidate_embeddings_saprot_np = candidate_embeddings_saprot_tensor.numpy()
        # 处理 NaN
        nan_mask_candidates = np.isnan(candidate_embeddings_saprot_np).any(axis=1)
        if np.any(nan_mask_candidates):
            num_nan_candidates = nan_mask_candidates.sum()
            print(f"  警告: 在候选嵌入中发现 {num_nan_candidates} 个 NaN 值。将移除它们。")
            original_count = len(candidate_list_saprot)
            valid_indices = ~nan_mask_candidates
            candidate_embeddings_saprot_np = candidate_embeddings_saprot_np[valid_indices]
            # 确保列表也使用相同的布尔索引或等效逻辑进行过滤
            candidate_list_saprot = [seq for i, seq in enumerate(candidate_list_saprot) if valid_indices[i]]
            mutation_list_saprot = [mut for i, mut in enumerate(mutation_list_saprot) if valid_indices[i]]
            print(f"  移除了 {original_count - len(candidate_list_saprot)} 个因 NaN 嵌入被移除的候选序列。")
            print(f"  剩余有效候选序列数量: {len(candidate_list_saprot)}")
            if candidate_embeddings_saprot_np.shape[0] > 0:
                print(f"  过滤后的候选嵌入形状: {candidate_embeddings_saprot_np.shape}")
            else:
                print("  过滤后没有剩余的有效候选嵌入。")
    else:
        print("  警告: 未能成功为候选序列生成 SaProt 嵌入。")
else:
    print("  上一步未能生成任何候选序列，跳过嵌入步骤。")

if candidate_embeddings_saprot_np.shape[0] == 0 and candidate_list_saprot:
    print("!!! 错误: 有候选序列但未能生成有效的嵌入。后续步骤将失败。")


# === 10. 预测候选序列的亮度 ===
print("\n[步骤 8/9] 正在使用基于 SaProt 的随机森林模型预测候选序列的亮度...")
predicted_brightness_saprot = [] # 保持初始化为列表
if rf_model_saprot is not None and candidate_embeddings_saprot_np.shape[0] > 0:
    if len(candidate_list_saprot) == candidate_embeddings_saprot_np.shape[0]:
        try:
            start_time_pred = time.time()
            # .predict() 返回 NumPy array
            predicted_brightness_saprot = rf_model_saprot.predict(candidate_embeddings_saprot_np)
            pred_duration = time.time() - start_time_pred
            print(f"  亮度预测完成，耗时 {pred_duration:.2f} 秒。")
            print(f"  成功预测了 {len(predicted_brightness_saprot)} 个候选序列的亮度。")
        except Exception as e:
            print(f"  使用 SaProt 随机森林模型进行预测时出错: {e}")
            predicted_brightness_saprot = [] # 出错时重置为空列表
    else:
        print(f"  错误: 候选序列列表 ({len(candidate_list_saprot)}) 与其嵌入 ({candidate_embeddings_saprot_np.shape[0]}) 数量不匹配。无法预测。")
elif rf_model_saprot is None:
     print("  随机森林模型未训练，无法预测。")
elif not candidate_list_saprot:
     print("  没有候选序列可供预测。")
elif candidate_embeddings_saprot_np.shape[0] == 0:
     print("  没有有效的候选序列 SaProt 嵌入可用于预测。")


# === 11. 过滤、选择 Top N 候选并保存结果 ===
print("\n[步骤 9/9] 正在过滤、选择 Top N 候选序列并保存结果...")
final_candidates_saprot_formatted = pd.DataFrame() # 初始化

# --- FIX: 使用 len() 检查 predicted_brightness_saprot ---
if candidate_list_saprot and len(predicted_brightness_saprot) > 0 and len(candidate_list_saprot) == len(predicted_brightness_saprot):
# --- End FIX ---
    results_saprot_df = pd.DataFrame({
        'Sequence': candidate_list_saprot,
        'Mutations': mutation_list_saprot,
        'PredictedBrightness_SaProt': predicted_brightness_saprot # NumPy array 可以直接放入DataFrame
    })
    print(f"  已创建包含 {len(results_saprot_df)} 个候选序列及其预测亮度的 DataFrame。")

    # 过滤排除列表
    exclusion_set = set(exclusion_sequences)
    initial_candidate_count = len(results_saprot_df)
    print(f"  正在根据排除列表 ({len(exclusion_set)} 条) 进行过滤...")
    results_saprot_df = results_saprot_df[~results_saprot_df['Sequence'].isin(exclusion_set)]
    removed_count = initial_candidate_count - len(results_saprot_df)
    if removed_count > 0: print(f"  移除了 {removed_count} 个在排除列表中的序列。")
    else: print("  候选序列均不在排除列表中。")
    print(f"  过滤后剩余候选序列: {len(results_saprot_df)} 条。")

    if not results_saprot_df.empty:
        results_saprot_df = results_saprot_df.sort_values(by='PredictedBrightness_SaProt', ascending=False)
        final_candidates_saprot = results_saprot_df.head(TOP_N_SELECT).copy()
        num_selected = len(final_candidates_saprot)
        print(f"\n  筛选出的 Top {num_selected} (最多 {TOP_N_SELECT}) 个 SaProt 预测候选 (已排除):")

        if num_selected > 0:
            final_candidates_saprot.insert(0, 'Sequence ID', [f'SaProt_Candidate_{i+1}' for i in range(num_selected)])
            final_candidates_saprot_formatted = final_candidates_saprot[['Sequence ID', 'Mutations', 'Sequence', 'PredictedBrightness_SaProt']]

            print("--- SaProt Top Candidates ---")
            display(final_candidates_saprot_formatted)
            print("----------------------------")

            submission_df_saprot = final_candidates_saprot_formatted[['Sequence ID', 'Mutations', 'Sequence']].copy()
            output_filename_saprot = "my_top_saprot_candidates.csv"
            try:
                submission_df_saprot.to_csv(output_filename_saprot, index=False)
                print(f"\n  已成功将 Top {len(submission_df_saprot)} 个 SaProt 候选序列保存到文件: {output_filename_saprot}")
            except Exception as e:
                print(f"\n  保存结果到 CSV 文件时出错: {e}")
        else:
            print("  经过滤和排序后，没有符合条件的 Top N SaProt 候选序列。")
    else:
        print("  根据排除列表过滤后，没有剩余的候选序列。")

# --- 更新这里的 elif 条件以匹配 if ---
elif not candidate_list_saprot:
     print("  由于没有生成或有效过滤候选序列，无法进行最终选择和保存。")
elif not (len(predicted_brightness_saprot) > 0): # 匹配 if 条件的检查方式
     print("  由于 SaProt 亮度预测失败或未产生结果 (predicted_brightness_saprot 为空)，无法进行最终选择和保存。")
else: # 这个分支对应 len(candidate_list_saprot) != len(predicted_brightness_saprot)
     print("  错误: 最终候选序列列表与预测结果数量不匹配。无法进行最终选择和保存。")


# === 12. 清理内存 (可选) ===
# print("\n--- 正在清理 SaProt 模型和相关张量以释放内存 ---")
# try:
#     # Safely delete variables if they exist
#     vars_to_delete = ['saprot_model', 'saprot_tokenizer', 'rf_model_saprot',
#                       'X_saprot_tensor', 'X_saprot', 'y_saprot',
#                       'X_train_saprot', 'y_train_saprot', 'X_val_saprot', 'y_val_saprot',
#                       'candidate_embeddings_saprot_tensor', 'candidate_embeddings_saprot_np',
#                       'results_saprot_df', 'final_candidates_saprot', 'final_candidates_saprot_formatted',
#                       'submission_df_saprot']
#     for var_name in vars_to_delete:
#         # Use try-except for deletion as variables might be local or global
#         try:
#             if var_name in locals(): del locals()[var_name]
#             if var_name in globals(): del globals()[var_name]
#         except NameError:
#             pass # Variable already deleted or never existed

#     # 如果使用了 GPU，清空 PyTorch 的 CUDA 缓存
#     if DEVICE_SAPROT == torch.device("cuda"):
#         print("  正在清空 CUDA 缓存...")
#         torch.cuda.empty_cache()
#     print("内存清理完成。")
# except Exception as e:
#     print(f"清理过程中出现错误: {e}")


print("\n--- SaProt 工作流程结束 ---")

HF_ENDPOINT 未设置或非预期值。将尝试设置 Hugging Face 端点为镜像: https://hf-mirror.com
当前脚本执行环境中 HF_ENDPOINT 已设置为: https://hf-mirror.com
--- 开始 SaProt 工作流程 ---
指定使用的 SaProt 模型 (来自镜像): westlake-repl/SaProt_35M_AF2
检测到 CUDA GPU，将使用 GPU 进行 SaProt 计算。
GPU 总显存: 7.86 GB
最终使用的 SaProt 设备: cuda
最终使用的 SaProt 模型: westlake-repl/SaProt_35M_AF2
最终使用的 SaProt 批次大小: 32

[步骤 1/9] 检查依赖变量/函数... 通过。

[步骤 2/9] 正在加载 SaProt Tokenizer 和模型: westlake-repl/SaProt_35M_AF2 到 cuda...
Some weights of the model checkpoint at westlake-repl/SaProt_35M_AF2 were not used when initializing EsmModel: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing EsmModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing EsmModel from the checkpoint o

,Sequence ID,Mutations,Sequence,PredictedBrightness_SaProt
0,SaProt_Candidate_1,C70I,MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,2.650868
328,SaProt_Candidate_2,S30Q:N105Q:M153P:V163E:T203S:L221D,MSKGEELFTGVVPILVELDGDVNGHKFSVQGEGEGDATYGKLTLKF...,2.650868
341,SaProt_Candidate_3,V68K:K101H:N105T:T203Y:L221V:H231E,MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,2.650868
340,SaProt_Candidate_4,S30F:F71G:K101M:N105S:M153T,MSKGEELFTGVVPILVELDGDVNGHKFSVFGEGEGDATYGKLTLKF...,2.650868
339,SaProt_Candidate_5,Y145A,MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,2.650868
338,SaProt_Candidate_6,G10F:N105I:Y145T:S147Y:I171P,MSKGEELFTFVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,2.650868


----------------------------

  已成功将 Top 6 个 SaProt 候选序列保存到文件: my_top_saprot_candidates.csv

--- SaProt 工作流程结束 ---


希望这个分步讲解能帮助大家理解整个流程！祝大家在蛋白质设计的探索中玩得开心，并在比赛中取得好成绩！✨